In [85]:
import os
import numpy as np
import pandas as pd
from path_origem_parquet import get_parquet_path
from path_destination_Dim import get_dim_path

path_parquet = os.path.join(get_parquet_path(), 'database.parquet')
destination_storeflag = os.path.join(get_dim_path(), 'dim_StoreFlag.parquet')

In [86]:
df = pd.read_parquet(path_parquet)
df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,7.2,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.0
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,5.8,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.0
5,2,2025-01-01 00:48:24,2025-01-01 01:08:26,2.0,2.63,1.0,N,239,68,2,19.1,1.0,0.5,0.00,0.0,1.0,24.10,2.5,0.0,0.0
6,1,2025-01-01 00:14:47,2025-01-01 00:16:15,0.0,0.40,1.0,N,170,170,1,4.4,3.5,0.5,2.35,0.0,1.0,11.75,2.5,0.0,0.0
7,1,2025-01-01 00:39:27,2025-01-01 00:51:51,0.0,1.60,1.0,N,234,148,1,12.1,3.5,0.5,2.00,0.0,1.0,19.10,2.5,0.0,0.0
8,1,2025-01-01 00:53:43,2025-01-01 01:13:23,0.0,2.80,1.0,N,148,170,1,19.1,3.5,0.5,3.00,0.0,1.0,27.10,2.5,0.0,0.0
9,2,2025-01-01 00:00:02,2025-01-01 00:09:36,1.0,1.71,1.0,N,237,262,2,11.4,1.0,0.5,0.00,0.0,1.0,16.40,2.5,0.0,0.0


In [87]:
df_dim_StoreFlag = df[['store_and_fwd_flag']].drop_duplicates().copy()

In [88]:
df_dim_StoreFlag.head(30)

,store_and_fwd_flag
0,N
92,Y
2935077,None


In [89]:
df_dim_StoreFlag = df_dim_StoreFlag.applymap(lambda x: 'U' if (pd.isna(x) or x == "") else x)

C:\Users\edmil\AppData\Local\Temp\ipykernel_10384\2908371382.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_dim_StoreFlag = df_dim_StoreFlag.applymap(lambda x: 'U' if (pd.isna(x) or x == "") else x)


In [ ]:
df_dim_StoreFlag.rename(columns={'store_and_fwd_flag': 'StoreAndFwdFlag'}, inplace=True)

In [ ]:
map_store_flag = {
    'Y': 'store and forward trip',
    'N': 'not a store and forward trip',
    'U': 'Null/unknown'
}

df_dim_StoreFlag['StoreFlagName'] = df_dim_StoreFlag['StoreAndFwdFlag'].map(map_store_flag)


In [92]:
df_dim_StoreFlag.head(30)

,StoreFwdFlag,StoreFlagName
0,N,not a store and forward trip
92,Y,store and forward trip
2935077,U,Null/unknown


In [ ]:
if os.path.exists(destination_storeflag):
    df_existente = pd.read_parquet(destination_storeflag)
    df_dim_StoreFlag = pd.concat([df_dim_StoreFlag, df_existente], ignore_index=True)
    
    # Tente tirar duplicados sem subset para evitar KeyError
    df_dim_StoreFlag = df_dim_StoreFlag.drop_duplicates()
else:
   pass

df_dim_StoreFlag.to_parquet(destination_storeflag, index=False)


In [96]:
# ler de teste ver se está ok no destino
df_test = pd.read_parquet(destination_storeflag)
df_test.head(20)

,StoreFwdFlag,StoreFlagName
0,N,not a store and forward trip
1,Y,store and forward trip
2,U,Null/unknown
